In [ ]:
## Machine Translation Seq2Seq

In [ ]:
import torch
if torch.cuda.is_available():
    device=torch.device(type='cuda', index=0)
else:
    device=torch.device(type='cpu', index=0)
print(device)

cuda:0


## **Major Steps**


*   Data Downloading
*   Data Loading(Custom Dataset,vocab)
*   Model Defination
*   Traning testing
*   Prediction  


In [ ]:
# @title
!pip install --upgrade torch torchtext

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_set_from_installed()
              

In [ ]:
# Data Downloading
train_url = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz"
valid_url = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz"

print("Downloading training data...")
!wget -q --show-progress {train_url}

print("\nDownloading validation data...")
!wget -q --show-progress {valid_url}

print("\nExtracting files...")
!tar -xzf training.tar.gz
!tar -xzf validation.tar.gz

print("\nExtraction complete. Resulting text files:")
!ls -l *.de *.en
# data to df
import pandas as pd
def create_translation_dataframe(german_filepath, english_filepath):
    try:
        with open(german_filepath, 'r', encoding='utf-8') as f:
            german_lines = [line.strip() for line in f.readlines()]

        with open(english_filepath, 'r', encoding='utf-8') as f:
            english_lines = [line.strip() for line in f.readlines()]

        if len(german_lines) != len(english_lines):
            print("Error: The number of lines in the two files does not match.")
            return None

        df = pd.DataFrame({
            'german': german_lines,
            'english': english_lines
        })

        return df

    except FileNotFoundError as e:
        print(f"Error: Could not find a file. Please check the path: {e.filename}")
        return None

train_df=create_translation_dataframe('train.de','train.en')
valid_df=create_translation_dataframe('val.de','val.en')

training.tar.gz.2   100%[===================>]   1.15M  --.-KB/s    in 0.04s   

validation.tar.gz.2 100%[===================>]  45.24K  --.-KB/s    in 0.007s  

Extracting files...

Extraction complete. Resulting text files:
-rw-rw-r-- 1 1000 1000 2110399 Feb  2  2016 train.de
-rw-rw-r-- 1 1000 1000 1801239 Feb  2  2016 train.en
-rw-rw-r-- 1 1000 1000   75920 Feb  2  2016 val.de
-rw-rw-r-- 1 1000 1000   63298 Feb  2  2016 val.en


In [ ]:
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 138.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 136.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy

de_nlp = spacy.load('de_core_news_sm')
en_nlp = spacy.load('en_core_web_sm')

def de_tokenizer(text):
    return [token.text for token in de_nlp.tokenizer(text)]

def en_tokenizer(text):
    return [token.text for token in en_nlp.tokenizer(text)]
en_tokenizer(train_df['english'][7])

UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

class Vocab:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.word2index = {'<unk>': 0, '<pad>': 1, '<bos>': 2, '<eos>': 3}
        self.index2word = {0: '<unk>', 1: '<pad>', 2: '<bos>', 3: '<eos>'}
        self.word2count = {}
        self.n_words = 4

    def build_vocab(self, sentence):
        for word in self.tokenizer(sentence):
            if word not in self.word2index:
                self.word2index[word] = self.n_words
                self.index2word[self.n_words] = word
                self.word2count[word] = 1
                self.n_words += 1
            else:
                self.word2count[word] += 1
vocab_de = Vocab(de_tokenizer)

for sentence in train_df['german']:
    vocab_de.build_vocab(sentence)

print(f"Built a German vocabulary with {vocab_de.n_words} words.")

vocab_en = Vocab(en_tokenizer)

for sentence in train_df['english']:
    vocab_en.build_vocab(sentence)

print(f"Built a English vocabulary with {vocab_en.n_words} words.")#

Built a German vocabulary with 19214 words.
Built a English vocabulary with 10837 words.


In [ ]:
import torch
from torch.utils.data import Dataset

class TranslationDataset(Dataset):
    def __init__(self, src_sentences, tgt_sentences, vocab_de, vocab_en):
        self.src_sentences = src_sentences
        self.tgt_sentences = tgt_sentences
        self.vocab_de = vocab_de
        self.vocab_en = vocab_en

    def __len__(self):
        return len(self.src_sentences)

    def __getitem__(self, idx):
        src_text = self.src_sentences[idx]
        tgt_text = self.tgt_sentences[idx]

        src_tokens = self.vocab_de.tokenizer(src_text)
        tgt_tokens = self.vocab_en.tokenizer(tgt_text)

        src_ids = [self.vocab_de.word2index.get(token, 0) for token in src_tokens]
        tgt_ids = [self.vocab_en.word2index.get(token, 0) for token in tgt_tokens]

        src_ids.append(self.vocab_de.word2index['<eos>'])
        tgt_ids.insert(0, self.vocab_en.word2index['<bos>'])
        tgt_ids.append(self.vocab_en.word2index['<eos>'])

        return torch.tensor(src_ids, dtype=torch.long), torch.tensor(tgt_ids, dtype=torch.long)

In [ ]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

PAD_IDX = vocab_de.word2index['<pad>']

def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(src_sample)
        tgt_batch.append(tgt_sample)

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX, batch_first=True)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX, batch_first=True)
    return src_batch, tgt_batch

train_dataset = TranslationDataset(train_df['german'].tolist(), train_df['english'].tolist(), vocab_de, vocab_en)
valid_dataset = TranslationDataset(valid_df['german'].tolist(), valid_df['english'].tolist(), vocab_de, vocab_en)

batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

In [ ]:
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, input_size, embed_size, hidden_size, dropout_p=0.1):
        super().__init__()
        self.e = nn.Embedding(input_size, embed_size)
        self.dropout = nn.Dropout(dropout_p)
        self.gru = nn.GRU(embed_size, hidden_size, batch_first=True)

    def forward(self, x):
        x = self.e(x)
        x = self.dropout(x)
        outputs, hidden = self.gru(x)
        return outputs, hidden

class Decoder(nn.Module):
    def __init__(self, output_size, embed_size, hidden_size):
        super().__init__()
        self.e = nn.Embedding(output_size, embed_size)
        self.relu = nn.ReLU()
        self.gru = nn.GRU(embed_size, hidden_size, batch_first=True)
        self.lin = nn.Linear(hidden_size, output_size)
        self.lsoftmax = nn.LogSoftmax(dim=-1)

    def forward(self, x, prev_hidden):
        x = self.e(x)
        x = self.relu(x)
        output, hidden = self.gru(x, prev_hidden)
        y = self.lin(output)
        y = self.lsoftmax(y)
        return y, hidden

In [ ]:
def train_one_epoch(encoder, decoder, opt_e, opt_d, loss_fn):
    encoder.train()
    decoder.train()
    track_loss = 0

    for i, (s_ids, t_ids) in enumerate(train_dataloader):
        s_ids = s_ids.to(device)
        t_ids = t_ids.to(device)

        encoder_outputs, encoder_hidden = encoder(s_ids)
        decoder_hidden = encoder_hidden

        yhats, decoder_hidden = decoder(t_ids[:, :-1], decoder_hidden)

        gt = t_ids[:, 1:]
        yhats_reshaped = yhats.view(-1, yhats.shape[-1])
        gt_reshaped = gt.reshape(-1)

        loss = loss_fn(yhats_reshaped, gt_reshaped)
        track_loss += loss.item()

        opt_e.zero_grad()
        opt_d.zero_grad()
        loss.backward()
        opt_e.step()
        opt_d.step()

    return track_loss / (i + 1)

def eval_one_epoch(encoder, decoder, loss_fn):
    encoder.eval()
    decoder.eval()
    track_loss = 0

    with torch.no_grad():
        for i, (s_ids, t_ids) in enumerate(valid_dataloader):
            s_ids = s_ids.to(device)
            t_ids = t_ids.to(device)

            encoder_outputs, encoder_hidden = encoder(s_ids)
            decoder_hidden = encoder_hidden

            yhats, decoder_hidden = decoder(t_ids[:, :-1], decoder_hidden)

            gt = t_ids[:, 1:]
            yhats_reshaped = yhats.view(-1, yhats.shape[-1])
            gt_reshaped = gt.reshape(-1)

            loss = loss_fn(yhats_reshaped, gt_reshaped)
            track_loss += loss.item()

    return track_loss / (i + 1)

In [ ]:
import torch.optim as optim

embed_size = 300
hidden_size = 512

encoder = Encoder(vocab_de.n_words, embed_size, hidden_size).to(device)
decoder = Decoder(vocab_en.n_words, embed_size, hidden_size).to(device)

loss_fn = nn.NLLLoss(ignore_index=PAD_IDX).to(device)
lr = 0.001
opt_e = optim.Adam(params=encoder.parameters(), lr=lr)
opt_d = optim.Adam(params=decoder.parameters(), lr=lr)

n_epochs = 10

for e in range(n_epochs):
    train_loss = train_one_epoch(encoder, decoder, opt_e, opt_d, loss_fn)
    eval_loss = eval_one_epoch(encoder, decoder, loss_fn)
    print(f"Epoch {e+1}/{n_epochs}, Train Loss: {train_loss:.4f}, Eval Loss: {eval_loss:.4f}")

Epoch 1/10, Train Loss: 3.8132, Eval Loss: 3.1398
Epoch 2/10, Train Loss: 2.7891, Eval Loss: 2.7676
Epoch 3/10, Train Loss: 2.2604, Eval Loss: 2.6332
Epoch 4/10, Train Loss: 1.8480, Eval Loss: 2.6013
Epoch 5/10, Train Loss: 1.5166, Eval Loss: 2.6123
Epoch 6/10, Train Loss: 1.2530, Eval Loss: 2.6496
Epoch 7/10, Train Loss: 1.0446, Eval Loss: 2.7120
Epoch 8/10, Train Loss: 0.8761, Eval Loss: 2.8066
Epoch 9/10, Train Loss: 0.7449, Eval Loss: 2.8886
Epoch 10/10, Train Loss: 0.6338, Eval Loss: 2.9766


In [ ]:
import torch

def translate_sentence(german_sentence, encoder, decoder, vocab_de, vocab_en, device, max_length=50):
    encoder.eval()
    decoder.eval()

    with torch.no_grad():
        src_tokens = vocab_de.tokenizer(german_sentence)
        src_ids = [vocab_de.word2index.get(token, 0) for token in src_tokens]
        src_ids.append(vocab_de.word2index['<eos>'])

        src_tensor = torch.tensor(src_ids, dtype=torch.long).unsqueeze(0).to(device)

        encoder_outputs, encoder_hidden = encoder(src_tensor)
        decoder_hidden = encoder_hidden

        decoder_input = torch.tensor([vocab_en.word2index['<bos>']], device=device)

        translated_ids = []

        for _ in range(max_length):
            probs, decoder_hidden = decoder(decoder_input.unsqueeze(1), decoder_hidden)
            _, top_id = torch.topk(probs, 1, dim=-1)
            predicted_id = top_id.squeeze().item()

            if predicted_id == vocab_en.word2index['<eos>']:
                break

            translated_ids.append(predicted_id)
            decoder_input = torch.tensor([predicted_id], device=device)

        translated_tokens = [vocab_en.index2word.get(idx, '<unk>') for idx in translated_ids]
        return " ".join(translated_tokens)
test_sentence = "Eine Gruppe von Männern lädt Baumwolle auf einen Lastwagen."

translation = translate_sentence(test_sentence, encoder, decoder, vocab_de, vocab_en, device)

print(f"Original (de): {test_sentence}")
print(f"Translated (en): {translation}")

Original (de): Eine Gruppe von Männern lädt Baumwolle auf einen Lastwagen.
Translated (en): A group of men are organizing a child on a truck .
